In [ ]:

#On ansible-master machine userdata

#!/bin/bash
yum install docker -y
systemctl enable docker.service
systemctl restart docker.service
sudo usermod -a -G docker ec2-user
amazon-linux-extras install ansible2 -y

In [ ]:
]# cat docker.vars 
---
site_url: "https://www.tooplate.com/zip-templates/2129_crispy_kitchen.zip"
site_dir: "2129_crispy_kitchen"
docker_port: 80



In [ ]:
# vim hosts

[amazon]

172.31.10.54 ansible_user="ec2-user" ansible_port=22 ansible_ssh_private_key_file="ansible.pem"


In [ ]:
# cat amazon-docker-nginx.yml 
---
- name: "Creating docker website on Amazon Linux"
  hosts: amazon
  become: yes
  vars_files:
    - docker.vars 

  tasks:
    - name: "Installing Docker on Amazon Linux"
      yum:
        name: 
          - docker
          - python-pip
        state: present
      register: docker_status

    - name: "python docker connector/module"
      pip:
        name: 
          - docker-py
        state: present

    - name: "Adding ec2-user to docker group"
      user:
        append: yes
        groups: docker
        name: ec2-user

    - name: "Restarting/enabling docker"
      when: docker_status.changed == true 
      service:
        name: docker 
        state: restarted
        enabled: yes

    - name: "Downloading website files"
      get_url:
        dest: "/home/ec2-user/"
        url: "{{ site_url }}"

    - name: "Extracting website files"
      unarchive:
        dest: "/home/ec2-user/"
        remote_src: yes
        src: "/home/ec2-user/{{ site_dir }}.zip"

    - name: "Create container"
      community.docker.docker_container:
        state: started
        detach: yes
        name: mycontainer1
        image: nginx:alpine
        restart_policy: always
        ports:
          - "{{ docker_port }}:80"
        volumes:
          - "/home/ec2-user/{{ site_dir }}:/usr/share/nginx/html"


In [ ]:
# ansible-playbook -i hosts amazon-docker-nginx.yml 

PLAY [Creating docker website on Amazon Linux] ************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
[WARNING]: Platform linux on host 172.31.10.54 is using the discovered Python interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/reference_appendices/interpreter_discovery.html for more information.
ok: [172.31.10.54]

TASK [Installing Docker on Amazon Linux] ******************************************************************************************************************
ok: [172.31.10.54]

TASK [python docker connector/module] *********************************************************************************************************************
ok: [172.31.10.54]

TASK [Adding ec2-user to docker group] ********************************************************************************************************************
ok: [172.31.10.54]

TASK [Restarting/enabling docker] *************************************************************************************************************************
skipping: [172.31.10.54]

TASK [Downloading website files] **************************************************************************************************************************
ok: [172.31.10.54]

TASK [Extracting website files] ***************************************************************************************************************************
ok: [172.31.10.54]

TASK [Create container] ***********************************************************************************************************************************
ok: [172.31.10.54]

PLAY RECAP ************************************************************************************************************************************************
172.31.10.54               : ok=7    changed=0    unreachable=0    failed=0    skipped=1    rescued=0    ignored=0